In [1]:
from cogent3 import load_aligned_seqs, get_app
import pathlib


In [ ]:
paths = list(pathlib.Path("../data/16s-10").glob("**/*.json"))

GN_model = get_app("model", sm = "GN", 
                unique_trees = True, time_het = "max", optimise_motif_probs = True, 
                show_progress = True, opt_args = dict(max_restarts=5))

no_degenerates = get_app(
    "omit_degenerates",
    moltype = None,
    gap_is_degen = True,
    motif_length = 1,
)

In [2]:
def alns_to_matx_pi(path, model):
    aln = load_aligned_seqs(path)
    aln1 = no_degenerates(aln)
    result = model(aln1)
    edge_names = result.tree.get_node_names(includeself = False)
    matrices = {n:result.lf.get_rate_matrix_for_edge(n, calibrated = True) for n in edge_names}
    motif_probs = result.lf.get_motif_probs()
    return matrices, motif_probs

matrix_dict_array = list()
motif_probs_dict_array = list()
for path in paths:
    matrix_dict_array.append(alns_to_matx_pi(paths[0], GN_model)[0])
    motif_probs_dict_array.append(alns_to_matx_pi(paths[0], GN_model)[1])


   0%|          |00:00<?

   0%|          |00:00<?

   0%|          |00:00<?

   0%|          |00:00<?

   0%|          |00:00<?

...

   0%|          |00:00<?

   0%|          |00:00<?

   0%|          |00:00<?

   0%|          |00:00<?

   0%|          |00:00<?

In [3]:
def process_compel_data(matrix_dict_array, motif_probs_dict_array):
    motif_probs_np_array = [motif_prob.array for motif_prob in motif_probs_dict_array]
    motif_probs_list = [inner_array.tolist() for inner_array in motif_probs_np_array]

    matrix_dict_np_array = [{n: matrix[n].array for n in matrix} for matrix in matrix_dict_array]
    matrix_dict_of_lists = [{k: v.tolist() for k, v in matrix.items()} for matrix in matrix_dict_np_array]

    data = dict()
    for i in range(len(matrix_dict_array)):
        data[i] = {'motif_prob': motif_probs_list[i], 'matrix': matrix_dict_of_lists[i]}

    return data

data = process_compel_data(matrix_dict_array, motif_probs_dict_array)

In [4]:
import json

with open('../data/matrix_motif.json', 'w') as json_file:
    json.dump(data, json_file)